<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       Complaints Analysis using Vantage and Azure OpenAI
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>

<p style = 'font-size:20px;font-family:Arial;color:#00233c'><b>Introduction:</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Analyzing consumer complaints using audio files conversations involves leveraging advanced technologies like large language models (LLM) and Azure OpenAI to extract insights from unstructured data. This process can be applied to various sectors, including financial services, telecommunications, and e-commerce, to improve customer experience and protect consumer rights.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233c'><b>Steps in the analysis:</b></p>
<ol style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li>Configuring the environment</li>
    <li>Configuring Azure OpenAI Model</li>
    <li>Speech Analysis</li>
</ol>

<hr style='height:2px;border:none;background-color:#00233C;'>
<b style = 'font-size:20px;font-family:Arial;color:#00233c'>1. Configuring the environment</b>
<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>1.1 Downloading and installing additional software needed</b>

In [ ]:
%%capture
!pip install -r requirements.txt --upgrade --quiet

<div class="alert alert-block alert-info">
<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Note: </b><i>Please restart the kernel after executing these two lines. The simplest way to restart the Kernel is by typing zero zero: <b> 0 0</b></i></p>

<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233c'><b>1.2 Import the required libraries</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Here, we import the required libraries, set environment variables and environment paths (if required).</p>

In [ ]:
# Data manipulation and analysis
import numpy as np
import pandas as pd

# Progress bar
from tqdm import tqdm

# User input
import getpass

# Teradata utilities
from teradataml import *

# OpenAI API
from openai import AzureOpenAI

# Requests
import requests

# Display settings
display.max_rows = 5
pd.set_option('display.max_colwidth', None)

<hr style="height:2px;border:none;background-color:#00233C;">
<b style='font-size:20px;font-family:Arial;color:#00233C'>2. Configuring Azure OpenAI</b>
<p style='font-size:16px;font-family:Arial;color:#00233C'>Before proceeding, you need to provide the following information:</p>
<ul style='font-size:16px;font-family:Arial;color:#00233C'>
<li><b>Endpoint</b>: Enter your Azure OpenAI deployment endpoint.</li>
<li><b>Whisper Endpoint</b>: Enter your Azure OpenAI deployment endpoint for Whisper model (Speech to Text)</li>
<li><b>Azure OpenAI API Key</b>: Enter your Azure OpenAI API Key.</li>
</ul>
<p style='font-size:16px;font-family:Arial;color:#00233C'>If you haven't retrieved your API Key and Endpoint yet, follow the instructions <a href="https://learn.microsoft.com/en-us/azure/ai-services/openai/quickstart?tabs=command-line%2Cpython-new&pivots=programming-language-python" target="_blank" style="color:#0066CC;text-decoration:none;"><b>here</b></a>.</p>
<p style='font-size:16px;font-family:Arial;color:#00233C'>Don't have an Azure OpenAI resource yet? Follow this guide:</p>
<a href="./Azure-OpenAI.ipynb" style="text-decoration:none;" target="_blank">
    <button style="font-size:16px;font-family:Arial;color:#fff;background-color:#00233C;border:none;border-radius:5px;cursor:pointer;height:50px;line-height:50px;display:flex;align-items:center;">
        Azure OpenAI Guide <span style="margin-left:10px;">&#8658;</span>
    </button>
</a>

In [ ]:
# Prompt user for Azure OpenAI endpoint securely
os.environ["ENDPOINT"] = getpass.getpass(prompt="\nPlease enter your Azure OpenAI endpoint: ")

# Prompt user for Azure OpenAI whisper model endpoint securely
os.environ["WHISPER_ENDPOINT"] = getpass.getpass(prompt="\nPlease enter your Azure OpenAI endpoint for whisper model: ")

# Prompt user for Azure OpenAI API key securely
os.environ["API_KEY"] = getpass.getpass(prompt="\nPlease enter your Azure OpenAI API key: ")

<hr style="height:1px;border:none;background-color:#00233C;">
<b style = 'font-size:18px;font-family:Arial;color:#00233C'>2.1 Initialize the Azure OpenAI API request</b>

In [ ]:
headers = {
    "Content-Type": "application/json",
    "api-key": os.environ["API_KEY"],
}

def get_payload(prompt):
    
    # Payload for the request
    payload = {
      "messages": [
        {
          "role": "system",
          "content": [
            {
              "type": "text",
              "text": "You are an assistant that categorizes text and gives reasoning for the categorization as well.\n"
            }
          ]
        },
        {
          "role": "user",
          "content": [
            {
              "type": "text",
              "text": prompt
            }
          ]
        }
      ],
      "temperature": 0.7,
      "top_p": 1,
      "max_tokens": 512
    }

    return(payload)

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>3. Speech Analysis</b>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The following cell defines a prompt that instructs the LLM on steps to be performed.</p>

In [ ]:
prompt = '''
    review: “{}”
    Analyze the given review to extract relevant information and categorize it accordingly. Follow these steps, providing reasoning for each task:

    Complaint Categorization: Determine whether the review is a complaint or a non-complaint. A complaint is typically a statement expressing dissatisfaction or grievance, while a non-complaint does not contain such elements. Explain why you categorized it this way.

    Topic Classification: If the review is a complaint, identify the most appropriate topic from the following list:
        Mortgage Application
        Payment Trouble
        Mortgage Closing
        Report Inaccuracy
        Payment Struggle
        Select only one topic based on the central issue or subject of the complaint. Explain the reasoning behind your choice of topic.

    Sentiment Analysis: Determine the sentiment of the review, which can be:
        Positive
        Negative
        Neutral
        Sentiment refers to the emotional tone or attitude conveyed in the review. Describe the reasoning behind the sentiment you identified.

    Summarization: Summarize the review in one sentence. A good summary should capture the essence of the review, highlighting the main point or issue. Explain the key elements that formed the basis of your summary.

    Return your results in the following format. Only keep the headings in bold:

    #### Complaint:
    - [Provide the raw complaint from the input.]

    #### Classification:
    - [Specify whether it's a Complaint or Non-Complaint]
      - **Reasoning with Chain-of-Thought**: [Explain the reasoning behind the classification.]

    #### Topic:
    - [Specify the relevant topic]
      - **Reasoning with Chain-of-Thought**: [Provide additional context or reasoning related to the topic.]

    #### Sentiment:
    - [Specify the sentiment (e.g., Positive, Negative, Neutral)]
      - **Reasoning with Chain-of-Thought**: [Explain the rationale for the sentiment.]

    #### Summary:
    - [Summarize the complaint in a concise sentence]
      - **Reasoning with Chain-of-Thought**: [Include any relevant reasoning or context for the summary.]
'''

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The following cell will transcribe the audio files using OpenAI <b>whisper-1</b> LLM model and call then call OpenAI <b>gpt-4o-mini</b> LLM to perform Complaint Analysis on it.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The Whisper model is a speech to text model from OpenAI that you can use to transcribe audio files. The model is trained on a large dataset of English audio and text. The model is optimized for transcribing audio files that contain speech in English. The model can also be used to transcribe audio files that contain speech in other languages. The output of the model is English text.</p>

In [ ]:
from IPython.display import display, Markdown, Audio, HTML

for file_name in sorted(os.listdir('./audio_files/')):
    audio_file = open(f"./audio_files/{file_name}", "rb")

    client = AzureOpenAI(
        api_key = os.environ["API_KEY"],
        api_version = "2024-02-01",
        azure_endpoint = os.environ["WHISPER_ENDPOINT"]
    )

    deployment_id = "whisper"

    result = client.audio.transcriptions.create(
        file = open(audio_file.name, "rb"),
        model = deployment_id
    )
    transcription = result.text
    
    # Send request
    try:
        response = requests.post(os.environ["ENDPOINT"], headers = headers, json = get_payload(prompt.format(transcription)))
        response.raise_for_status()
    except requests.RequestException as e:
        raise SystemExit(f"Failed to make the request. Error: {e}")

    output = response.json()['choices'][0]['message']['content']

    # Description HTML content
    desc = f'''<hr style="height:1px;border:none;background-color:#00233C;">
               <p style='font-size:18px;font-family:Arial;color:#00233C'><b>For file: {file_name}</b></p>'''

    # Create the HTML, audio, and markdown objects
    desc_display = HTML(desc)
    audio_display = Audio(f"./audio_files/{file_name}")
    markdown_display = Markdown(output)

    # Display all elements in a single output
    display(desc_display, audio_display, markdown_display)

<footer style="padding-bottom:35px; background:#f9f9f9; border-bottom:3px solid #00233C">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2024. All Rights Reserved
        </div>
    </div>
</footer>